In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import laplace
import torch.nn as nn
import torch.nn.functional as F
from backpack import extend
from laplace.curvature import BackPackGGN, AsdlHessian

In [2]:
%cd ../..

/nfs/homedirs/fuchsgru/MastersThesis


In [3]:
from configuration import ExperimentConfiguration, update_with_default_configuration
from model.build import make_model
from data.build import load_data_from_configuration
from data.util import data_get_num_attributes, data_get_num_classes
import data.constants as dconstants
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
from model.prediction import Prediction
from torch.utils.data import TensorDataset

In [4]:
from model.nn import GCNConv

class LaplaceGCN(nn.Module):
    
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.conv1 = GCNConv(num_inputs, 64)
        self.conv2 = GCNConv(64, 64)
        self.head = nn.Linear(64, num_outputs)
        
    def forward_feature_extractor(self, batch):
        x, edge_index = batch.x, batch.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return x
        
    def forward(self, batch, feature_extraction=False):
        h = self.forward_feature_extractor(batch)
        if not feature_extraction:
            return self.head(h)
        else:
            return h
    

In [9]:
lgcn = LaplaceGCN(
        data_get_num_attributes(datasets[dconstants.TRAIN][0]), 
        data_get_num_classes(datasets[dconstants.TRAIN][0])
)
lgcn

LaplaceGCN(
  (conv1): GCNConv(2879, 64)
  (conv2): GCNConv(64, 64)
  (head): Linear(in_features=64, out_features=7, bias=True)
)

In [10]:
# Make a tensor dataset
with torch.no_grad():
    data_train = datasets[dconstants.TRAIN][0]
    data_val = datasets[dconstants.VAL][0]
    h = lgcn(data_train, feature_extraction=True)
    laplace_data_train = TensorDataset(h[data_train.mask], data_train.y[data_train.mask])
    laplace_data_val = TensorDataset(h[data_val.mask], data_val.y[data_val.mask])

In [11]:
laplace_data_loader_train = torch.utils.data.DataLoader(laplace_data_train, batch_size=len(laplace_data_train), shuffle=False)
laplace_data_loader_val = torch.utils.data.DataLoader(laplace_data_val, batch_size=len(laplace_data_val), shuffle=False)

In [12]:
la = laplace.Laplace(
    lgcn.head,
    'classification',
     subset_of_weights='all',
     hessian_structure='diag',
)
la.fit(laplace_data_loader_train)
la.optimize_prior_precision(method='CV', val_loader=laplace_data_loader_val)

In [16]:
l1 = la(h, )
l2 = la(h, )

In [17]:
(l1 == l2).all()

tensor(True)

In [25]:
la.predictive_samples(h, n_samples=10).std(0).size()

torch.Size([2810, 7])

In [54]:
from torch.utils.data import TensorDataset

In [12]:
y = torch.randint(3, (1000,))
x = torch.randn(y.size(0), 64)
x += torch.exp(y)[:, None]
x *= 0.2

dataset = TensorDataset(x, y)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32)

NameError: name 'TensorDataset' is not defined

In [13]:
class Model(nn.Module):
    
    def __init__(self, indim, outdim):
        super().__init__()
        self.lin1 = nn.Linear(indim, 64)
        self.lin2 = nn.Linear(64, 64)
        self.out = nn.Linear(64, outdim)
        
    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.relu(x)
        return self.out(x)
        

In [10]:
la = laplace.Laplace(
    Model(
        x.size(1),
        torch.max(y) + 1,
    ),
    'classification',
     subset_of_weights='last_layer',
     hessian_structure='diag',
)
la.fit(data_loader)

NameError: name 'Model' is not defined

In [11]:

la.optimize_prior_precision(method='CV', val_loader=data_loader)

NameError: name 'la' is not defined

In [76]:
la

In [6]:
config = ExperimentConfiguration(data={'dataset' : 'cora_ml'}, run={'use_default_configuration' : True})
update_with_default_configuration(config)

In [7]:
config.data

DataConfiguration(dataset='cora_ml', train_portion=20, val_portion=20, test_portion_fixed=0.2, base_labels='all', train_labels='all', corpus_labels='all', left_out_class_labels=[], max_attempts_per_split=5, drop_train_vertices_portion=0.1, setting='transductive', ood_type='perturbations', ood_sampling_strategy='all', split_type='uniform', type='npz', perturbation_budget=0.1, min_token_frequency=10, preprocessing='none', language_model='bert-base-uncased', normalize='l2', vectorizer='tf-idf', integrity_assertion=True)

In [8]:
datasets, _ = load_data_from_configuration(config.data, 1337)

In [88]:
class LaplaceApproximationDataLoaderWrapper:
    
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
        
    def __iter__(self):
        self._it = 0
        return self

    def __next__(self):
        if self._it < len(self.dataset):
            data = self.dataset[self._it]
            self._it += 1
            return data.x[None, :, :], data.y[data.mask]
        else:
            raise StopIteration
            
    def __len__(self):
        return len(self.dataset)

In [89]:
data_loaders = {k : LaplaceApproximationDataLoaderWrapper(data) for k, data in datasets.items()}

In [93]:
from model.nn import GCNConv

class LaplaceGCN(nn.Module):
    
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.conv1 = GCNConv(num_inputs, 64)
        #self.conv2 = GCNConv(64, 64)
        self.head = nn.Linear(64, num_outputs)
        self.edge_index = None
        self.mask = None
        
    def set_edge_index_and_mask(self, edge_index, mask):
        self.edge_index = edge_index.clone()
        self.edge_index.requires_grad = False
        self.mask = mask.clone()
        self.mask.requires_grad = False
        
    def forward(self, x):
        x = x.squeeze()
        print(x.size(), self.edge_index.max())
        x = self.conv1(x, self.edge_index)
        #x = F.relu(x)
        #x = self.conv2(x, edge_index)
        #x = F.relu(x)
        out = self.head(x)
        return out[self.mask]
    


la_feature_extractor = laplace.utils.FeatureExtractor(
    LaplaceGCN(
        data_get_num_attributes(datasets[dconstants.TRAIN][0]), 
        data_get_num_classes(datasets[dconstants.TRAIN][0])
    )
)

In [94]:
lgcn = LaplaceGCN(
        data_get_num_attributes(datasets[dconstants.TRAIN][0]), 
        data_get_num_classes(datasets[dconstants.TRAIN][0])
    )
lgcn.set_edge_index_and_mask(
    data_loaders[dconstants.TRAIN].dataset[0].edge_index,
    data_loaders[dconstants.TRAIN].dataset[0].mask,
)


la = laplace.Laplace(
    lgcn, 
    'classification',
     subset_of_weights='last_layer',
     hessian_structure='diag',
)


In [95]:
la.fit(data_loaders[dconstants.TRAIN])

torch.Size([2810, 2879]) tensor(2809)
torch.Size([2810, 2879]) tensor(2809)
torch.Size([2810, 2879]) tensor(2809)


AssertionError: BackPACK extension expects a backpropagation quantity but it is None. Module: Linear(in_features=64, out_features=7, bias=True), Extension: <backpack.extensions.secondorder.diag_ggn.DiagGGNExact object at 0x7f94aca60dc0>.

In [30]:

la.optimize_prior_precision(method='CV', val_loader=val_loader)

NameError: name 'val_loader' is not defined

In [7]:
model = make_model(config, data_get_num_attributes(datasets[dconstants.TRAIN][0]), 
                data_get_num_classes(datasets[dconstants.TRAIN][0])) 

In [8]:
model

SemiSupervisedNodeClassification(
  (backbone): GCN(
    (convs): ModuleList(
      (0): BasicBlock(
        (conv): GCNConv(2879, 64)
        (act): LeakyReLU(negative_slope=0.01)
      )
      (1): BasicBlock(
        (conv): GCNConv(64, 7)
      )
    )
  )
)

In [30]:
class ConvertibleGCN(nn.Module):
    
    def __init__(self, n, d, h, c):
        super().__init__()
        self.A1 = nn.Linear(n, n)
        self.lin1 = nn.Linear(d, h)
        self.A2 = nn.Linear(n, n)
        self.lin2 = nn.Linear(h, c)
        
    def forward(self, x):
        out = self.lin1(x)
        out = self.A1(out.T).T
        out = F.leaky_relu(out)
        out = self.lin2(out)
        out = self.A2(out.T).T
        return out
        

In [31]:
ConvertibleGCN(4, 5, 64, 3)(torch.randn(4, 5)).size()

torch.Size([4, 3])

In [32]:
model = extend(ConvertibleGCN(100, 1771, 64, 7), use_converter=True)

AttributeError: 'builtin_function_or_method' object has no attribute 'split'

In [10]:
bb = extend(model.backbone, use_converter=True)

ValueError: `MessagePassing.propagate` only supports `torch.LongTensor` of shape `[2, num_messages]` or `torch_sparse.SparseTensor` for argument `edge_index`.

In [94]:
class LaplaceApproximationWrapper(nn.Module):
    
    def __init__(self, base_model, base_data):
        super().__init__()
        self.base_model = base_model
        self.base_data = base_data
    
    def forward(self, X):
        # Ignore x all together
        pred: Prediction = self.base_model(self.base_data)
        logits = pred.get_logits(average=True)[self.base_data.mask]
        return logits
        
    

In [91]:
for x, y in data_loaders[dconstants.TRAIN]:
    print(y.size())

torch.Size([140])


In [103]:
la = laplace.Laplace(extend(LaplaceApproximationWrapper(model, datasets[dconstants.TRAIN][0])), 'classification',
             subset_of_weights='all',
             hessian_structure='diag',
                    backend=AsdlHessian)

In [104]:
la.fit(data_loaders[dconstants.TRAIN])
la.optimize_prior_precision(method='CV', val_loader=val_loader)


NotImplementedError: 

In [ ]:

# User-specified predictive approx.
pred = la(x, link_approx='probit')